In [ ]:
# still trying to construct the for-loop used to generate the total nC2 similarity scores 
# where n is the total number of recipes

In [93]:
import pandas as pd
import gensim
import spacy
nlp = spacy.load("en_core_web_lg")

In [94]:
df1 = pd.read_csv('kawaling_pinoy.csv')
df1.head()

,url,food,prep time,cook time,ingredients,instructions
0,https://www.kawalingpinoy.com/ginisang-pusit/,Ginisang Pusit,10 mins,20 mins,"2 pounds squid, 1/2 cup vinegar, 4 cloves garl...",Hold the tail tube portion of the squid and wi...
1,https://www.kawalingpinoy.com/polvoron/,Polvoron,10 mins,20 mins,"2 cups flour, 1 cup powdered full cream milk (...","In a skillet over medium heat, add flour. Toas..."
2,https://www.kawalingpinoy.com/ube-maja-blanca/,Ube Maja Blanca,10 mins,40 mins,"2 cups coconut cream, 2 cans (13.5 ounces each...","In a pan over medium heat, add coconut cream a..."
3,https://www.kawalingpinoy.com/adobong-talaba/,Adobong Talaba,10 mins,15 mins,"1 tablespoon canola oil, 2 tablespoons butter,...","In a pan over medium heat, heat oil. Add butte..."
4,https://www.kawalingpinoy.com/miswa-at-patola-...,Miswa at Patola Soup,10 mins,20 mins,"3 medium patola, 1 tablespoon canola oil, 1 on...",Cut off both ends of the patola and discard. U...


In [95]:
df = df1[["food","ingredients"]].copy()
df = pd.DataFrame(df)
df

,food,ingredients
0,Ginisang Pusit,"2 pounds squid, 1/2 cup vinegar, 4 cloves garl..."
1,Polvoron,"2 cups flour, 1 cup powdered full cream milk (..."
2,Ube Maja Blanca,"2 cups coconut cream, 2 cans (13.5 ounces each..."
3,Adobong Talaba,"1 tablespoon canola oil, 2 tablespoons butter,..."
4,Miswa at Patola Soup,"3 medium patola, 1 tablespoon canola oil, 1 on..."
...,...,...
589,Garlic Butter Fried Frog Legs,"1 lb about 3 to 4 pieces frog legs, 1 tablespo..."
590,Tinolang Manok,"1 tablespoon canola oil, 1 small onion, peeled..."
591,Tilapia in Black Bean Garlic Sauce,"4 (4 ounces each) tilapia fillets, salt and pe..."
592,Pork Adobo,"2 pounds pork belly, cut into 2-inch cubes, 1 ..."


In [96]:
df = df.reset_index()

In [97]:
data = df.rename(columns = {"index":"Id", "ingredients": "text"})
data

,Id,food,text
0,0,Ginisang Pusit,"2 pounds squid, 1/2 cup vinegar, 4 cloves garl..."
1,1,Polvoron,"2 cups flour, 1 cup powdered full cream milk (..."
2,2,Ube Maja Blanca,"2 cups coconut cream, 2 cans (13.5 ounces each..."
3,3,Adobong Talaba,"1 tablespoon canola oil, 2 tablespoons butter,..."
4,4,Miswa at Patola Soup,"3 medium patola, 1 tablespoon canola oil, 1 on..."
...,...,...,...
589,589,Garlic Butter Fried Frog Legs,"1 lb about 3 to 4 pieces frog legs, 1 tablespo..."
590,590,Tinolang Manok,"1 tablespoon canola oil, 1 small onion, peeled..."
591,591,Tilapia in Black Bean Garlic Sauce,"4 (4 ounces each) tilapia fillets, salt and pe..."
592,592,Pork Adobo,"2 pounds pork belly, cut into 2-inch cubes, 1 ..."


In [98]:
ref_sent = data.loc[data['Id'] == 590, "text"].iloc[0]
ref_sent

'1 tablespoon canola oil, 1 small onion, peeled and sliced thinly, 3 cloves garlic, peeled and minced, 2 thumb-sized fresh ginger, 1 (3 to 4 pounds) whole chicken, cut into serving pieces, 2 tablespoons fish sauce, 5 cups water, 1 small green papaya, pared, seeded and cut into 2-inch wedges, 1 bunch fresh spinach leaves, stems trimmed, salt and pepper to taste'

In [99]:
ref_sent_vec = nlp(ref_sent) #Vectorize tokens

In [100]:
all_docs = [nlp(row) for row in data['text']]
all_docs

[2 pounds squid, 1/2 cup vinegar, 4 cloves garlic, peeled and minced, 1/2 teaspoon peppercorns, cracked, 1 bay leaf, 2 tablespoons canola oil, 1 onion, peeled and thinly sliced, 2 large Roma tomatoes, chopped, 1 tablespoon fish sauce, salt and pepper to taste,
 2 cups flour, 1 cup powdered full cream milk (Nido), 1 cup sugar, 1 cup butter, melted, 1/2 cup pinipig, 1/2 cup unsalted cashew or peanut, 6 Oreo cookies, 1 tablespoon butter, melted,
 2 cups coconut cream, 2 cans (13.5 ounces each) coconut milk, 1 can (14 ounces) sweetened condensed milk, 1 can (12 ounces) evaporated milk, 2 cups cooked ube, mashed, 3/4 cup sugar, 4 drops ube extract, 1/2 cup water, 3/4 cup cornstarch,
 1 tablespoon canola oil, 2 tablespoons butter, 1 onion, peeled and chopped, 4 cloves garlic, peeled and minced, 2 thumb size ginger, peeled and julienned, 2 pounds oysters, (meat only), drained well, 1/4 cup soy sauce, pepper to taste, 1/4 cup calamansi juice, salt to taste,
 3 medium patola, 1 tablespoon canol

In [101]:
sims = []
doc_id = []
for i in range(len(all_docs)):
    sim = all_docs[i].similarity(ref_sent_vec)
    sims.append(sim)
    doc_id.append(i)
    sims_docs = pd.DataFrame(list(zip(doc_id,sims)), columns = ['doc_id', 'sims'])
    

In [113]:
sims_docs_sorted = sims_docs.sort_values(by = 'sims', ascending = False)
sims_docs_sorted

,doc_id,sims
590,590,1.000000
473,473,0.991544
467,467,0.991473
10,10,0.990170
110,110,0.989977
...,...,...
460,460,0.777131
52,52,0.774293
407,407,0.773746
120,120,0.771871


In [114]:
top5_sim_docs = data.iloc[sims_docs_sorted['doc_id'][1:6]]
top5_sim_docs

,Id,food,text
473,473,Tinolang Manok sa Upo,"1 tablespoon canola oil, 1 onion, peeled and s..."
467,467,Pinatisang Manok,"1 tablespoon canola oil, 1 onion, peeled and s..."
10,10,Picadillo,"1 tablespoon canola oil, 1 small onion, peeled..."
110,110,Kinamatisang Baboy,"1 tablespoon canola oil, 1 onion, peeled and c..."
555,555,Chicken Halang Halang,"2 stalks lemongrass, 1 tablespoon canola oil, ..."


In [119]:
# print(data[data['Id'] == 590]['text'].values)

In [116]:
recipe = data.iloc[590]['food'] #data[data['Id'] == 590]['food'].values
print(recipe)

Tinolang Manok


In [120]:
top_sim_scores = pd.concat([top5_sim_docs, sims_docs_sorted['sims'][1:6]], axis = 1)
top_sim_scores

,Id,food,text,sims
473,473,Tinolang Manok sa Upo,"1 tablespoon canola oil, 1 onion, peeled and s...",0.991544
467,467,Pinatisang Manok,"1 tablespoon canola oil, 1 onion, peeled and s...",0.991473
10,10,Picadillo,"1 tablespoon canola oil, 1 small onion, peeled...",0.990170
110,110,Kinamatisang Baboy,"1 tablespoon canola oil, 1 onion, peeled and c...",0.989977
555,555,Chicken Halang Halang,"2 stalks lemongrass, 1 tablespoon canola oil, ...",0.989921


In [118]:
print(f"The top 5 similar recipes based on ingredients for {recipe} are:\n")
for (i, recipe, text, sim) in zip(range(1,6), top_sim_scores['food'], top_sim_scores['text'], top_sim_scores['sims']):
    print(f"#{i} {recipe}: \n{text} \nSimilarity Score: {sim:.5f}\n")
    i = i+1

The top 5 similar recipes based on ingredients for Tinolang Manok are:

#1 Tinolang Manok sa Upo: 
1 tablespoon canola oil, 1 onion, peeled and sliced thinly, 3 cloves garlic, peeled and minced, 2 thumb-sized ginger, peeled and julienned, 2 pounds bone-in chicken, cut into 3-inch pieces, 2 tablespoons fish sauce, 4 cups water, 1 large upo, peeled and cut into 2-inch wedges (about 2 cups), 1 bunch fresh spinach leaves, stems trimmed, salt and pepper to taste 
Similarity Score: 0.99154

#2 Pinatisang Manok: 
1 tablespoon canola oil, 1 onion, peeled and sliced thinly, 2 cloves garlic, peeled and minced, 2 thumb-size ginger, peeled and julienned, 1 whole (about 3 to 4 pounds) chicken, cut into serving parts, 3 tablespoons fish sauce, 2 cups water, 1 hardboiled egg yolk, salt and pepper to taste, 2 cups fresh chili leaves 
Similarity Score: 0.99147

#3 Picadillo: 
1 tablespoon canola oil, 1 small onion, peeled and chopped, 2 cloves garlic, peeled and minced, 2 pounds chuck roast, cut into 1